In [1]:
import ast
import json
import nltk
import pandas as pd

#nltk.download('punkt')

with open('data/recipes_raw_nosource_ar.json', 'r') as f:
    recipes1 = json.load(f)

In [7]:
class Recipe:
    def __init__(self, uid, r):
        self.measures = ['bunch', 'block', 'cup', 'cups', 'pinch', 'tablespoon', 'tablespoons', 'ounces', 'ounce', 'oz',
                         'teaspoon', 'teaspoons', 'tsp', 'pound', 'pounds', 'sheets', 'slice', 'wheel', 'bag', 'bags',
                         'gram', 'grams', 'sticks', 'ml', 'mL', 'kg']
        self.uid = uid
        self.shopping_list = []
        self.instructions = r['instructions']
        self.ingredients = {x: {} for x in r['ingredients']}
        if 'title' in r:
            self.title = r['title']
            
    # strip and lower

    def ingredients_list(self):
        for entry in self.ingredients:
            clean_entry = entry.replace('-', ' ').replace('/', ' ').lower()
            tokens = nltk.word_tokenize(clean_entry)
            pos = nltk.pos_tag(tokens)
            #print(pos)
            unit = [x[0] for x in pos if x in self.measures]
            qty = [x[0] for x in pos if x[1] == 'CD' or x[1] == 'NNP']
            item = [x[0] for x in pos if x[1] == 'NN' and x[0] not in self.measures or x[1] =='NNS' and x[0] not in self.measures]
            notes = [x for x in tokens if x not in qty and x not in item]
            self.ingredients[entry]['qty'] = ' '.join(qty)
            self.ingredients[entry]['ingredient'] = ' '.join(item)
            self.ingredients[entry]['notes'] = ' '.join(notes)
            self.shopping_list.append(' '.join(item))
            

In [8]:
class CookBook:
    def __init__(self, recipes):
        self.recipes = recipes
        self.ingredients = []
        self.invalid = 0

        
    def get_ingredients(self):
        for r in self.recipes.keys():
            if 'ingredients' not in self.recipes[r] or 'instructions' not in self.recipes[r]:
                self.invalid += 1
                continue
            else:
                rec = Recipe(r, self.recipes[r])
                rec.ingredients_list()
                self.ingredients = self.ingredients  + rec.shopping_list
        print("INVALID COUNT: ", self.invalid)
        return self.ingredients

    #def to_dict(self):
        

In [ ]:

cb = CookBook(recipes)
shopping_list = pd.Series(cb.get_ingredients()).unique()
print(shopping_list)

In [69]:
from copy import deepcopy
s = deepcopy(shopping_list)
print(len(s))
for i in range(32050, 32100):
    print(s[i])

80489
pieces leaf lettuce
peanut butter jif® crunchy peanut butter
butter piece greasing
butterscotch semisweet chocolate peanut butter chocolate chips
raisin bread loaf
leeks ramps scallions
castor sugar
oil spray pan
pea sauce recipe
sauteed pea shoots recipe
package crusts
tbsp cranberries
tbsp pecans
vinaigrette salad
kosher potato starch pan
slices quality dense wheat bread
chili paste chili paste
nonpareil capers
mini buns
cucumber yogurt sauce recipe
tin anchovies
romaine hearts heads hearts
pears lengthwise halves thirds
nectarines quarters
figs halves
medjool dates lengthwise
sesame fries
sesame bits crackers
ground beef pork mixture
poblano peppers seeds ribs
gold potatoes skins inch cubes
piece puff inches inches
flour salt pepper sage
pepper inches
scraps trimmings
madiera
carrot onion celery trimmings
dark rum appleton estate
beef kidney inch pieces
parsley stalks inch pieces
mushrooms inch slices
oysters juice
rib rib roast
bobby flay spice rub pork beef recipe
margarita 

In [150]:
fish = ['anchovies', 'barracuda', 'basa', 'bass', 'cod', 'sablefish', 'blowfish', 'bluefish',
        'bombay duck', 'bream', 'brill', 'butter fish', 'catfish', 'dogfish', 'dorade', 'eel',
        'flounder', 'grouper', 'haddock', 'hake', 'halibut', 'herring', 'ilish', 'john dory', 'lamprey',
        'lingcod', 'mackerel', 'mahi mahi', 'monkfish', 'mullet', 'orange roughy', 'parrotfish', 'patagonian toothfish',
        'perch', 'pike', 'pilchard', 'pollock', 'pomfret', 'pompano', 'sablefish', 'salmon', 'sanddab', 'sardine',
        'sea bass', 'shad', 'alewife', 'shark', 'skate', 'smelt', 'snakehead', 'snapper', 'rockfish', 'sole', 'sprat', 
        'sturgeon', 'surimi', 'swordfish', 'tilapia', 'tilefish', 'trout', 'tuna', 'albacore', 'yellowfin', 'bigeye',
        'bluefin', 'dogtooth', 'turbot', 'wahoo', 'whitefish', 'whiting', 'witch', 'righteye', 'whitebait', 'roe', 'caviar',
        'ikura', 'kazunoko', 'lumpfish','masago', 'capelin', 'tobiko', 'flying-fish', 'shellfish', 'crustaceans', 'crab',
        'dungeness', 'crayfish', 'langostino', 'lobster', 'shrimp', 'prawns', 'mollusca', 'cockle', 'cuttlefish', 'clam',
        'loco', 'mussel', 'octopus', 'oyster', 'periwinkle', 'scallop', 'squid', 'escargot', 'snails', 'nautilus', 'dolphin',
        'seal', 'whale', 'echinoderms', 'sea cucumber', 'uni', 'sea urchin', 'roe', 'medusozoa', 'Tunicates', 'sea squirts',
        'microcosmus', 'sabatieri', 'sea fig', 'sea violet', 'crabmeat']

meat = ['chicken', 'turkey', 'duck', 'mallard', 'goose','greylag', 'guinea', 'fowl', 'pigeon', 'dove', 'quail',
        'partridge', 'swiftlet', 'swan', 'beef', 'steak', 'sirloin', 'veal', 'brisket', 'pork', 'ham', 'bacon', 'shank', 'lamb',
        'mutton', 'goat', 'chuck']

keyword = ['meat', 'seafood', 'vegetarian', 'vegan']


In [88]:
with open('train.json/train.json', 'r') as f:
    nationalities = json.load(f)

In [92]:
class DietModel:
    def __init__(self, training, ratios):
        self.training = training
        self.model = {c: {'ingredients': {},
                          'total_items': 0,
                          'ratio': ratios[c]['ratio'] * 100000} for c in ratios}
        self.model['world'] = {'ingredients': {},
                               'total_items': 0,
                               'ratio': 100000}
        
    def build_model(self):
        for item in self.training:
            if item['cuisine'] not in self.model:
                self.model[item['cuisine']] = {}
            for i in item['ingredients']:
                if i not in self.model['world']['ingredients']:
                    self.model['world']['ingredients'][i] = 1
                    self.model['world']['total_items'] += 1
                else:
                    self.model['world']['ingredients'][i] +=1
                if i not in self.model[item['cuisine']]['ingredients']:
                    self.model[item['cuisine']]['ingredients'][i] = 1
                    self.model[item['cuisine']]['total_items'] += 1
                else:
                    self.model[item['cuisine']]['ingredients'][i] +=1
                    
    def clean_up(self):
        for region in self.model:
            clean = []
            for item in self.model[region]['ingredients']:
                if self.model[region]['ingredients'][item] <= 2:
                    clean.append(item)
            for item in clean:
                del self.model[region]['ingredients'][item]
                self.model[region]['total_items'] -=1
                    
    

In [93]:
print(len(nationalities))

countries = {}

for item in nationalities:
    if item['cuisine'] not in countries:
        countries[item['cuisine']] = {'count':1}
    else:
        countries[item['cuisine']]['count'] += 1

sum = 0

for country in countries:
    countries[country]['ratio'] = countries[country]['count']/39774 
    sum+= countries[country]['ratio']
    print(country, ": ", countries[country])
    
    

39774
greek :  {'count': 1175, 'ratio': 0.02954191180167949}
southern_us :  {'count': 4320, 'ratio': 0.10861366721979182}
filipino :  {'count': 755, 'ratio': 0.018982249710866397}
indian :  {'count': 3003, 'ratio': 0.07550158394931362}
jamaican :  {'count': 526, 'ratio': 0.013224719666113542}
spanish :  {'count': 989, 'ratio': 0.024865490018605118}
italian :  {'count': 7838, 'ratio': 0.19706340825665009}
mexican :  {'count': 6438, 'ratio': 0.16186453462060643}
chinese :  {'count': 2673, 'ratio': 0.06720470659224619}
british :  {'count': 804, 'ratio': 0.020214210288127924}
thai :  {'count': 1539, 'ratio': 0.03869361894705084}
vietnamese :  {'count': 825, 'ratio': 0.020742193392668577}
cajun_creole :  {'count': 1546, 'ratio': 0.038869613315231054}
brazilian :  {'count': 467, 'ratio': 0.011741338562880274}
french :  {'count': 2646, 'ratio': 0.0665258711721225}
japanese :  {'count': 1423, 'ratio': 0.03577714084577865}
irish :  {'count': 667, 'ratio': 0.01676974908231508}
korean :  {'count'

In [94]:
food_model = DietModel(nationalities, countries)
food_model.build_model()
food_model.clean_up()

for c in food_model.model:
    print('-----------------------------')
    print(c, ": ")
    for ingredient in food_model.model[c]:
        print(ingredient, ": ", food_model.model[c][ingredient])
        


-----------------------------
greek : 
ingredients :  {'romaine lettuce': 39, 'black olives': 31, 'grape tomatoes': 26, 'garlic': 216, 'pepper': 203, 'purple onion': 186, 'seasoning': 3, 'garbanzo beans': 23, 'feta cheese crumbles': 252, 'ground pork': 4, 'finely chopped fresh parsley': 8, 'onions': 185, 'salt': 572, 'vinegar': 3, 'minced garlic': 68, 'dried oregano': 267, 'red wine vinegar': 99, 'olive oil': 504, 'lemon juice': 183, 'orange': 12, 'anise': 5, 'cinnamon sticks': 31, 'orange blossom honey': 3, 'sugar': 77, 'lemon': 129, 'clove': 14, 'honey': 67, 'plain whole-milk yogurt': 5, 'fresh dill': 108, 'yoghurt': 10, 'large eggs': 72, 'cheese': 15, 'feta cheese': 191, 'phyllo': 12, 'kefalotyri': 9, 'ground black pepper': 221, 'extra-virgin olive oil': 229, 'hamburger buns': 6, 'paprika': 20, 'chopped fresh mint': 68, 'ground cinnamon': 73, 'balsamic vinegar': 24, 'baby spinach leaves': 9, 'ground lamb': 68, 'dried mint flakes': 15, 'tomatoes': 156, 'dried dillweed': 10, 'red wine

In [95]:
count = 0
for food in s:
    guess = 0
    food = food.split()
    for f in food:
        if f in food_model.model['world']['ingredients']:
            guess +=1
    if guess == 0:
        print(food)
        count +=1
print(count)

['inch', 'onion']
['jalapenos', 'medium']
[]
['cloves']
['onion']
['plum', 'tomato']
['chile']
['cheddar']
['sweet', 'peppers']
['peel']
['radish', 'inches']
['strips']
['carrot']
['bell', 'peppers']
['currants']
['roasting', 'chickens']
['potato']
['powder']
['lb', 'velveeta', 'product', 'inch', 'cubes']
['inch', 'london', 'broil']
['baking', 'powder']
['egg']
['bbq']
['jalapenos']
['onion', 'powder']
['bonnet', 'peppers']
['laurel', 'leaf']
['leek', 'part']
['piece', 'parmesan']
['egg', 'yolks']
['limes']
['drops', 'food', 'coloring']
['acid', 'crystals']
['tomato', 'paste']
['sprigs']
['medium', 'onion']
['fleur', 'sel']
['ribs']
['onion', 'cut', 'pieces']
['inch', 'thick', 'prime', 'porterhouse', 'steaks']
['onion', 'medium', 'onion']
['marzano', 'tomato', 'puree']
['mayonnaise']
['anchovy']
['cans', 'hearts']
['feta']
['wedges', 'recipe']
['breads']
['pizza', 'margherita', 'recipe']
['pecorino', 'romano']
['medium', 'size', 'jicamas']
['oranges']
['piquin', 'chile', 'powder']
['li

['pickle', 'spear']
['cube', 'cheddar']
['rings', 'onion']
['mozzarella', 'buffalo']
['potato', 'inch', 'cubes']
['apple', 'inch', 'cubes']
['package', 'almond', 'brickle', 'bits']
['chile', 'arbol', 'powder']
['spicy', 'mayonnaise']
['jalapeno', 'chile', 'spicy']
['hellmann', 'mayonnaise']
['muffuletta']
['inch', 'thick', 'slices']
['bricks', 'cast', 'iron', 'pan', 'cans']
['rings', 'chunks']
['quarts']
['container', 'spread']
['hearts', 'romaine', 'lengthwise']
['pink']
['ratio']
['whitener']
['pink', 'food', 'coloring', 'chefmaster']
['backs']
['bass', 'fillet']
['jalapeno', 'peppers', 'heat']
['level', 'tomato', 'puree']
['mature', 'cheddar']
['popovers', 'recipe']
['chops', 'bones']
['prune', 'pulp']
['bonnet', 'chile']
['queso', 'blanco']
['scallion', 'slices']
['carrot', 'julienne', 'strips']
['shallot', 'cloves']
['inamona', 'nut', 'seed', 'candlenut', 'tree']
['food', 'coloring', 'colors']
['pans', 'parchment', 'foil']
['bran', 'flakes']
['package', 'potato', 'pieces']
['inch'

['fructose']
['package', 'fettuccine', 'substitute', 'house', 'foods']
['maple', 'flakes']
['jan', 'spice', 'recipe']
['mushroom', 'caps', 'cremini', 'clean']
['pita', 'toasts']
['peppers', 'season']
['onion', 'coarse']
['parmigiana']
['store', 'home']
['queso', 'anejo', 'parmesan']
['quart', 'clam']
['bran']
['lb', 'snapper', 'head', 'tail', 'intact']
['mix', 'note']
['cooking', 'spray', 'pans']
['drop', 'jellies']
['string', 'licorice', 'twizzlers', 'peel']
['bell', 'peppers', 'inch', 'cubes']
['bell', 'peppers', 'inch', 'squares']
['metal', 'wood', 'skewers']
['package', 'breast', 'tenders']
['reggiano', 'romano', 'table']
['head', 'celeriac']
['chile', 'paste', 'grocery', 'stores']
['string', 'cheese*']
['rabbits']
['handful', 'currants']
['cheeks']
['cloves', 'paste', 'flat', 'knife']
['broil', 'inches']
['peels']
['cucumbers', 'inch', 'slices', 'diagonal']
['jalapeno', 'chilies']
['heads', 'cloves', 'half', 'lengthwise']
['pearl']
['bones', 'bass']
['package', 'wonton', 'wrappers

['leaf', 'cook', 'note*']
['chickens', 'eighths']
['buttercream', 'recipe', 'cupcakes']
['eats', 'mix', 'recipe']
['bratwurst', 'ground']
['entree']
['haricots', 'verts', 'thirds']
['leftover', 'inch', 'cheesecake']
['toffee', 'brickle']
['pinches', 'powder']
['matzo', 'meal*']
['cornstarch', 'table']
['equipment', 'inch', 'pizza', 'pan', 'pizza', 'stone']
['scallion', 'diagonal']
['sausage', 'sweet', 'spicy']
['provolone', 'slices']
['chunk', 'halloumi']
['package', 'breads', 'plain', 'pieces']
['mozzarella', 'pieces*']
['testa']
['inch', 'breads', 'pockets']
['tri', 'bell', 'peppers']
['crispy', 'onion', 'strings', 'recipe']
['shallot*']
['jalapeno', 'peppers', 'rubber', 'gloves']
['ear', 'cloves']
['rromano']
['bourbon', 'brushing']
['bunches', 'rabe', 'split']
['ripe', 'peach']
['frozen', 'bell', 'stir', 'fry', 'mix', 'peppers']
['hothouse', 'cucumbers', 'medium', 'dice']
['recipe', 'fermentee']
['packages', 'snack', 'almond', 'size', 'pieces']
['leek', 'parts', 'lengthwise', 'inch

['smith', 'apple', 'concasse']
['cucumbers', 'lengthwise', 'inch', 'pieces']
['medium', 'bell', 'peppers', 'lengthwise', 'cores']
['medium', 'eggplants', 'lengthwise', 'inch', 'thick', 'slices']
['pretzels', 'rods', 'half']
['lb', 'bob', 'sausage', 'roll']
['monterey', 'paste']
['baby', 'carrot', 'mandoline']
['box', 'fudge', 'brownie', 'mix']
['hellmann', 'dijonnaise']
['jalapeno', 'taste']
['tbs', 'nectar']
['inamona', 'kukui', 'nut']
['honeycrisp', 'mcintosh', 'apple']
['toasts']
['jar', 'mole']
['tortilla', 'recipe']
['medium', 'onion', 'wedges', 'inch', 'wedges']
['¿2']
['cans', 'scungili', 'conch', 'dry']
['slices', 'cut', 'thicker', 'side', 'cheddar']
['mini', 'inch', 'pita', 'pockets', 'pita']
['carrot', 'top']
['carob']
['challah', 'loaf', 'inch', 'thick']
['jalapenos', 'slices', 'dog']
['cucumbers', 'kirbies', 'skins']
['onion', 'width']
['style', 'mix']
['grain', 'loaf', 'inch', 'cubes']
['bottles', 'malta', 'ingredient', 'others']
['laurel', 'leaf', 'bay', 'leaf', 'bay', 's

['package', 'breadsticks']
['cheddar', 'pieces']
['inch', 'onion', 'middle']
['inch', 'round', 'pie', 'crusts']
['banana', 'lengthways']
['inch', 'cubes', 'cheddar']
['pear', 'nectar']
['chow', 'chow', 'recipe']
['package', 'instructions']
['onion', 'sauteed']
['inch', 'store', 'pie', 'doughs']
['parsnip', 'inch']
['sweet', 'potato', 'inch']
['bottle', 'malta']
['curd', 'room', 'temperature']
['avocados', 'inch', 'thick']
['box', 'fudge', 'brownie', 'mix', 'betty', 'crocker']
['package', 'shoots']
['bull', 'energy', 'drink']
['paprika*']
['purple', 'bell', 'peppers', 'strips']
['slices', 'onion', 'inch', 'thick']
['johnnie', 'walker', 'label']
['woody', 'ends', 'bias']
['espellette']
['avocados', 'ripe', 'service']
['tissue', 'paper']
['pots', 'sizes']
['flower', 'foam']
['flower', 'cookie', 'cutters']
['tape']
['store', 'colors']
['colors', 'food', 'coloring']
['florida', 'avocados']
['containers', 'hunt']
['cucumbers', 'seedless', 'hothouse']
['beet', 'degrees', 'hour', 'minutes']
['

['skinless', 'fillet', 'strips']
['yellowtail', 'fillet', 'strips']
['cheddar', 'brick']
['jiggers', 'liquer']
['pear', "d'anjou", 'inch', 'slices']
['package', 'strawberry', 'mousse']
['pre', 'cannoli']
['broil', 'round']
['carrot', 'slices']
['mozzarella', 'ball', 'strips']
['lobsters', 'freezer', 'minutes', 'hour', 'cooking']
['scallion', 'angle']
['sausage', 'strips', 'inches']
['quality', 'chunks']
['cracker', 'pieces', 'broken']
['head', 'cloves', 'cloves']
['witch', 'brew', 'recipe']
['frying', 'chickens']
['cool']
['piece', 'chiffon']
['spicy', 'sriracha', 'mayo', 'recipe']
['beefsteak', 'heirloom', 'tomato']
['band', 'bream']
['onion', 'dice', 'inch']
['gold', 'maine', 'pei', 'potato', 'dice', 'inch']
['yogurt', 'fage', 'percent']
['medium', 'onion', 'lengthwise', 'end', 'intact']
['vadouvan', 'madras', 'powder']
['rocoto', 'chile', 'paste']
['strawberry', 'stem']
['jp', 'spice', 'mix', 'recipe']
['equipment', 'inch', 'pan']
['barolo']
['oranges', 'peels', 'cut', 'off', 'segme

['bean', 'sprout']
['d']
['inch', 'thick', 'pickle', 'slices']
['medium', 'dice', 'onion']
['perch']
['candies', 'dots', 'n', 'plenty', 'whoppers']
['package', 'nut', 'halves']
['plain', 'yogurt', 'nonfat']
['mozzarella', 'part', 'skim']
['center', 'mignon', 'steaks']
['cheddar', 'room', 'temperature']
['mayonnaise', 'room', 'temperature']
['hoisin', 'baste', 'recipe']
['medium', 'jicima', 'inch', 'squares']
['jalepeno', 'chilies']
['splash', 'tabasco']
['pizzelles']
['egg', 'yokes']
['onion', 'cut', 'chunks']
['julienne', 'strips']
['box', 'butterscotch', 'mix', 'jell', 'o']
['buffalo', 'ribs', 'inches']
['bonnets', 'habaneros']
['grill', 'palm']
['caramel', 'variations', 'recipes']
['dungeness', 'crabs', 'lbs']
['inch', 'hero', 'bun']
['scallion', 'half', 'green']
['jarslberg']
['onion', 'portions']
['mayonnaise', 'home']
['skinless', 'fillet', 'cook', 'note']
['food', 'coloring', 'marker']
['cooking', 'spray', 'baking', 'mat']
['gum', 'balls']
['tomato', 'paste', 'tomato', 'paste']
